# Capstone Project: NYC Taxi Fare Prediction 🚕

**Gabriele Cola** - **Swann Etro**

### 0. Libraries

In [1]:
# Importing data
import pandas as pd
import geopandas as gpd
import numpy as np
import google.cloud.bigquery

# Stats
from scipy.stats import zscore
from statsmodels.tsa.stattools import adfuller


# Data Visualization
import seaborn as sns
import folium
from folium.plugins import HeatMap, MarkerCluster

import plotly
import plotly.express as px

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches

import shapely

# Modelling
import sklearn as sk
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV,   RandomizedSearchCV
from sklearn.metrics import mean_squared_error,r2_score,silhouette_score
from sklearn.decomposition import PCA
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.cluster import KMeans, DBSCAN

# Save model
import joblib

### 1. Importing

In [2]:
from google.cloud import bigquery
client = bigquery.Client(project='nyc-taxi-project-455608')

In [ ]:
query= """
SELECT *
 FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022`
 ORDER BY RAND()
 LIMIT 100000
"""

df = client.query(query).to_dataframe()
df.head()

/Users/gabrielecola/anaconda3/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1933: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,imp_surcharge,airport_fee,total_amount,pickup_location_id,dropoff_location_id,data_file_year,data_file_month
0,1,2022-03-12 00:13:07+00:00,2022-03-12 00:21:43+00:00,0,1.500000000,1.0,N,1,8.000000000,3.000000000,0.500000000,1.000000000,0E-9,0.300000000,0E-9,12.800000000,142,48,2022,3
1,2,2022-05-16 15:04:47+00:00,2022-05-16 15:14:36+00:00,3,1.250000000,1.0,N,1,8.500000000,0E-9,0.500000000,2.000000000,0E-9,0.300000000,0E-9,13.800000000,238,236,2022,5
2,2,2022-06-03 15:10:12+00:00,2022-06-03 15:22:13+00:00,1,1.700000000,1.0,N,1,9.500000000,0E-9,0.500000000,2.560000000,0E-9,0.300000000,0E-9,15.360000000,43,239,2022,6
3,2,2022-09-13 19:58:46+00:00,2022-09-13 20:08:26+00:00,1,1.960000000,1.0,N,1,8.500000000,1.000000000,0.500000000,2.560000000,0E-9,0.300000000,0E-9,15.360000000,141,137,2022,9
4,1,2022-10-21 19:45:02+00:00,2022-10-21 19:56:26+00:00,1,2.100000000,1.0,N,1,10.500000000,3.500000000,0.500000000,1.500000000,0E-9,0.300000000,0E-9,16.300000000,142,238,2022,10


### 2. Pre-Processing

#### 2.1 Changing Type of Features

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype              
---  ------               --------------   -----              
 0   vendor_id            100000 non-null  object             
 1   pickup_datetime      100000 non-null  datetime64[ns, UTC]
 2   dropoff_datetime     100000 non-null  datetime64[ns, UTC]
 3   passenger_count      96574 non-null   Int64              
 4   trip_distance        100000 non-null  object             
 5   rate_code            96574 non-null   object             
 6   store_and_fwd_flag   96574 non-null   object             
 7   payment_type         100000 non-null  object             
 8   fare_amount          100000 non-null  object             
 9   extra                100000 non-null  object             
 10  mta_tax              100000 non-null  object             
 11  tip_amount           100000 non-null  object             
 12  tol

In [5]:
# Converto tutte le colonne che dovrebbero essere numeriche ma sono 'object'
cols_to_convert = ['trip_distance', 'fare_amount', 'extra', 'mta_tax',
                   'tip_amount', 'tolls_amount', 'imp_surcharge',
                   'airport_fee', 'total_amount']

for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce') # mette Nan in caso di errore

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype              
---  ------               --------------   -----              
 0   vendor_id            100000 non-null  object             
 1   pickup_datetime      100000 non-null  datetime64[ns, UTC]
 2   dropoff_datetime     100000 non-null  datetime64[ns, UTC]
 3   passenger_count      96574 non-null   Int64              
 4   trip_distance        100000 non-null  float64            
 5   rate_code            96574 non-null   object             
 6   store_and_fwd_flag   96574 non-null   object             
 7   payment_type         100000 non-null  object             
 8   fare_amount          100000 non-null  float64            
 9   extra                100000 non-null  float64            
 10  mta_tax              100000 non-null  float64            
 11  tip_amount           100000 non-null  float64            
 12  tol

#### 2.2 Checking NA

In [7]:
df.isnull().sum()

vendor_id                 0
pickup_datetime           0
dropoff_datetime          0
passenger_count        3426
trip_distance             0
rate_code              3426
store_and_fwd_flag     3426
payment_type              0
fare_amount               0
extra                     0
mta_tax                   0
tip_amount                0
tolls_amount              0
imp_surcharge             0
airport_fee            3426
total_amount              0
pickup_location_id        0
dropoff_location_id       0
data_file_year            0
data_file_month           0
dtype: int64

In [8]:
def mean_impute_columns(df, columns):
    for col in columns:
        if df[col].dtype == 'object':
            # Per colonne categoriche: riempio con la modalità più frequente
            df[col] = df[col].fillna(df[col].mode().iloc[0] if not df[col].mode().empty else 'Unknown')
        elif pd.api.types.is_numeric_dtype(df[col]):
            # Per colonne numeriche
            mean_val = df[col].dropna().mean()
            if pd.notna(mean_val):
                df[col] = df[col].fillna(round(mean_val))
            else:
                df[col] = df[col].fillna(0)  # Se non esiste una media, riempio con 0 o un valore predefinito
    return df

In [9]:
df = mean_impute_columns(df, ['passenger_count', 'rate_code','store_and_fwd_flag','airport_fee'])

In [10]:
df.isnull().sum()

vendor_id              0
pickup_datetime        0
dropoff_datetime       0
passenger_count        0
trip_distance          0
rate_code              0
store_and_fwd_flag     0
payment_type           0
fare_amount            0
extra                  0
mta_tax                0
tip_amount             0
tolls_amount           0
imp_surcharge          0
airport_fee            0
total_amount           0
pickup_location_id     0
dropoff_location_id    0
data_file_year         0
data_file_month        0
dtype: int64

#### 2.3 Removing Outlier

In [11]:
# Pulizia della colonna trip distance da outlier
# Calcola Q1, Q3 e l'IQR per trip_distance
Q1 = df['trip_distance'].quantile(0.01)
Q3 = df['trip_distance'].quantile(0.99)
IQR = Q3 - Q1

# Definisci i limiti per i valori validi
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

# Filtra i dati per rimuovere gli outlier
df = df[(df['trip_distance'] >= lower_limit) & (df['trip_distance'] <= upper_limit)]

# Mostra il risultato
df[['trip_distance']].describe()

,trip_distance
count,99987.000000
mean,3.480990
std,4.370424
min,0.000000
25%,1.100000
50%,1.900000
75%,3.540000
max,49.970000


### 3. EDA

In [12]:
describe_df = df.describe().transpose().round(2)
# Aggiunta di una colonna con le unità di misura
unit_dict = {
    'trip_distance': 'miles',
    'fare_amount': '$',
    'extra': '$',
    'mta_tax': '$',
    'tip_amount': '$',
    'tolls_amount': '$',
    'imp_surcharge': '$',
    'airport_fee': '$',
    'total_amount': '$',
    'passenger_count': 'people',
    'data_file_year': 'year',
    'data_file_month': 'month'
}

describe_df['measurment units'] = describe_df.index.map(unit_dict).fillna('-')  # '-' per le colonne senza unità definite

# Visualizza
describe_df.reset_index().rename(columns={"index": "variabile"})

,variabile,count,mean,std,min,25%,50%,75%,max,measurment units
0,passenger_count,99987.0,1.38743,0.952389,0.0,1.0,1.0,1.0,6.0,people
1,trip_distance,99987.0,3.48099,4.370424,0.0,1.1,1.9,3.54,49.97,miles
2,fare_amount,99987.0,14.544054,13.651209,-160.0,7.0,10.0,16.0,495.0,$
3,extra,99987.0,0.996295,1.2474,-4.5,0.0,0.5,2.5,8.25,$
4,mta_tax,99987.0,0.488846,0.093227,-0.5,0.5,0.5,0.5,2.78,$
5,tip_amount,99987.0,2.710175,3.113477,-6.87,0.93,2.16,3.35,100.0,$
6,tolls_amount,99987.0,0.522477,1.951922,-20.3,0.0,0.0,0.0,39.75,$
7,imp_surcharge,99987.0,0.295952,0.048681,-0.3,0.3,0.3,0.3,0.3,$
8,airport_fee,99987.0,0.0943,0.333658,-1.25,0.0,0.0,0.0,1.25,$
9,total_amount,99987.0,21.280387,17.099991,-169.85,12.3,15.95,22.8,499.18,$
